In [ ]:
import lib_process
from collections import Counter

In [ ]:
import config
import psycopg2
conn = psycopg2.connect(config.DB_PROD)
cur = conn.cursor()

In [ ]:
cur.execute("SELECT id, text FROM records WHERE customer_id = 2 AND processed = -5 AND DATE(source_created_at) > '2016-11-05' LIMIT 300")
list_from_db = cur.fetchall()

In [ ]:
def get_top_objects(rlist):
    cntr = Counter()
    for line in rlist:
        obj_list = lib_process.get_objects(line[1], strmode = False, check_whitelist = False)
        cntr.update(obj_list)
    return cntr

In [ ]:
%%time
top_bad = get_top_objects(list_from_db)

In [ ]:
top_bad.most_common(200)

In [ ]:
cur.execute("SELECT id, name FROM topics where customer_id = 2")
topics = cur.fetchall()

In [ ]:
topic_ids = []
topic_names = {}
for t in topics:
    topic_ids.append(str(t[0]))
    topic_names[t[0]] = t[1]

In [ ]:
cur.execute("SELECT id, name, topic_id FROM keywords where topic_id in ({0})".format(','.join(topic_ids)))
keywords = cur.fetchall()

In [ ]:
topic_keywords = {}
for k in keywords:
    t_id = k[2]
    if not t_id in topic_keywords:
        topic_keywords[t_id] = []
    topic_keywords[t_id].append(str(k[0]))

In [ ]:
for topic_id in topic_keywords:
    cur.execute("SELECT record_id FROM record_keywords WHERE keyword_id in ({0})".format(','.join(topic_keywords[topic_id])))
    list_from_db = cur.fetchall()
    record_ids = []
    for line in list_from_db:
        record_ids.append(str(line[0]))
    cur.execute("SELECT id, text, objects FROM records WHERE processed = 1 AND customer_id=7 AND DATE(source_created_at) > '2016-10-12' AND DATE(source_created_at) < '2016-10-21' AND id IN ({0})".format(','.join(record_ids)))
    list_from_db = cur.fetchall()
    cntr = Counter()
    for line in list_from_db:
        cntr.update(line[2])
    print('total', len(list_from_db))
    print(cntr.most_common(50))

In [ ]:
cntr = 0
cntr_good = 0
for line in list_from_db:
    cntr += 1
    if cntr % 100 == 0:
        print('>', cntr, cntr_good)
    record_id = line[0]
    text = line[1]
    for keyword_line in keywords:
        keyword_id = keyword_line[0]
        keyword_line_text = keyword_line[1]
        has_all = True
        for k in keyword_line_text.split(' '):
            if not k in text:
                has_all = False
                break
        if has_all:
            try:
                cur.execute("INSERT INTO record_keywords (record_id, keyword_id) VALUES (%s, %s)", (record_id, keyword_id))
                cntr_good += 1 
                break
            except:
                pass

In [ ]:
%%time
cnt = 0
for row in list_from_db:
    new_objects = lib_process.get_objects(row[1], strmode=False)
    objects_str = '{' + ','.join(new_objects) + '}'
    cur.execute("UPDATE records SET objects = '{0}', processed=1 WHERE id={1}".format(objects_str, row[0]))
    cnt += 1
    if cnt % 10 == 0:
        print(cnt)

In [ ]:
cur.close()
conn.close()